In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import os
import sys
import json
import copy
import pickle
from hashlib import sha256
import random

In [3]:
import pandas as pd
import pandas as pd
import numpy as np
import cv2
from tqdm.notebook import tqdm

In [4]:
sys.path.append('../lared')
from helpers import file_functions
from dataset.example import VideoExample, AudioExample, FullExample
from constants.laughter_constants import data_path, cloud_data_path
from data_loading.utils import get_video_caps, AudioFiles

In [5]:
VideoExample.init_caps(get_video_caps('/mnt/hdd/data/lared/concat'))

In [6]:
laughter_ann_path = os.path.join(cloud_data_path, 'laughter.txt')
camera_ann_path = os.path.join(cloud_data_path, 'camera.txt')
out_path = os.path.join(cloud_data_path, 'examples')
images_path = os.path.join(out_path, 'images')


laughter_ann = pd.read_csv(laughter_ann_path, sep='\t',names=['name','participant','ini','fin','dur','type'],header=None)
camera_ann = pd.read_csv(camera_ann_path, sep='\t',names=['name','participant','ini','fin','dur','camera'],header=None)

In [7]:
camera_ann.head()

,name,participant,ini,fin,dur,camera
0,camera30,32.0,0.000,10.000,10.000,0
1,camera3,3.0,866.280,882.440,16.160,1
2,camera3,3.0,883.150,884.490,1.340,12
3,camera3,3.0,884.600,888.860,4.260,123
4,camera3,3.0,4300.139,4300.956,0.817,4


In [8]:
df_camera_by_participant = camera_ann.groupby('participant')
df_laughter_by_participant = laughter_ann.groupby('participant')

In [9]:
dfs_camera = []
for i in df_camera_by_participant.groups:
    dfs_camera.append(df_camera_by_participant.get_group(i))
dfs_laughter = []
for i in df_laughter_by_participant.groups:
    dfs_laughter.append(df_laughter_by_participant.get_group(i))
participant_ids = df_camera_by_participant.groups.keys()
participant_ids = [int(id) for id in participant_ids]

In [10]:
def get_camera(camerastr):
    if int(camerastr) == 0:
        return []
    
    return [int(c) for c in str(camerastr)]

In [11]:
# find the camera where a person is seen
def find_cameras(cam, t):
    previous = cam[cam['fin'] < t]
    if len(previous) == 0:
        return []
    
    row = previous.iloc[-1]
    cameras = row.tail()['camera']
    return get_camera(cameras)

# Laughter examples

In [12]:
# Create examples and first images
examples = dict()
for pid, df_laughter, df_camera in zip(participant_ids, dfs_laughter, dfs_camera):
    for index,row in df_laughter.iterrows():
        cameras = find_cameras(df_camera, row['ini'])
        ini_time = (row['ini'])
        fin_time = (row['fin'])

        for cam in cameras:

            hashstr = '_'.join([str(pid), str(cam), str(ini_time), str(fin_time)])
            hashid = sha256(hashstr.encode()).hexdigest()
            
            examples[hashid] = FullExample({
                'id': hashid,
                'pid': pid,
                'cam': cam,
                'ini_time': ini_time,
                'fin_time': fin_time
            })

            examples[hashid].write_full_first_image(images_path)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [12]:
def create_random_segments(df_camera, pid, segment_len=10):
    examples = list()
    it = df_camera.iterrows()
    prev = next(it)[1]
    for index, row in it:
        ini = prev.loc['fin'] + 10
        fin = row.loc['ini'] - 10
        cameras = get_camera(prev.loc['camera'])

        for cam in cameras:
            t = ini
            while t < fin:
                if t < 14*60:
                    t += segment_len
                    continue

                hashstr = '_'.join([str(pid), str(cam), str(t), str(t+segment_len)])
                hashid = sha256(hashstr.encode()).hexdigest()
                examples.append(FullExample({
                    'id': hashid,
                    'pid': pid,
                    'cam': cam,
                    'ini_time': t,
                    'fin_time': t + segment_len
                }))
                t += segment_len
        prev = row
    return examples

In [ ]:
create_random_segments()

In [13]:
random_examples = [create_random_segments(df, pid) for pid, df in zip(participant_ids, dfs_camera)]

In [14]:
random_examples = [ex for e in random_examples for ex in e]

In [15]:
# sample examples and put them in order of pid (for easy labeling)
random.seed(22)
chosen_examples = random.sample(random_examples, 2000)
chosen_examples = sorted(chosen_examples, key=lambda ex: ex.pid)

In [16]:
len(chosen_examples)

2000

In [17]:
chosen_examples[-1]

{'id': 'ee3f09b742090254947c8e97915ee096b67f4e79ddc69da4f78c95a1c3fe102d', 'pid': 35, 'cam': 3, 'valid': None, 'ini_time': 3177.52, 'fin_time': 3187.52, 'labels': {'vad': None, 'label': None, 'vad_seg': None}, 'rect': None, 'ini': None, 'len': None}

In [31]:
chosen_examples[1200]

{'id': '388e20647366104e822959a67a7ef67cde0d1ca80e928928e0f6487cd07ff0d6', 'pid': 17, 'cam': 4, 'valid': None, 'ini_time': 5191.6759999999995, 'fin_time': 5201.6759999999995, 'labels': {'vad': None, 'label': None, 'vad_seg': None}, 'rect': None, 'ini': None, 'len': None}

In [33]:
chosen_examples[1200].write_full_video('./')

# Write images

In [24]:
for ex in tqdm(chosen_examples[:2000]):
    ex.write_full_first_image(images_path)

KeyboardInterrupt: 